# To analyse two MUTUAL FUND portfolios
    
   + Compare mautual fund investments
   + Identify new investments made and sell offs

In [11]:
import pandas as pd
import numpy as np
import re
import datetime
import logging
import glob
from collections import OrderedDict

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [12]:
files = glob.glob("data/*.xls")

In [13]:
month_pat = re.compile(r'january|february|march|april|may|june|july|august|september|october|november|december')
year_pat = re.compile(r'20\d\d')

In [14]:
for f in files:
    print(f)
    month = re.findall(month_pat,f)
    year = re.findall(year_pat,f)
    print(month,year)

data\monthly-portfolio-as-on-29th-february-2020.xls
['february'] ['2020']
data\monthly-portfolio-as-on-30th-april-2019.xls
['april'] ['2019']
data\monthly-portfolio-as-on-30th-june-2019.xls
['june'] ['2019']
data\monthly-portfolio-as-on-30th-november-2019.xls
['november'] ['2019']
data\monthly-portfolio-as-on-31-october-2019.xls
['october'] ['2019']
data\monthly-portfolio-as-on-31st-august-2019.xls
['august'] ['2019']
data\monthly-portfolio-as-on-31st-december-2019.xls
['december'] ['2019']
data\monthly-portfolio-as-on-31st-january-2020.xls
['january'] ['2020']
data\monthly-portfolio-as-on-31st-july-2019-new.xls
['july'] ['2019']
data\monthly-portfolio-as-on-31st-march-2019.xls
['march'] ['2019']
data\monthly-portfolio-as-on-31st-may-2019.xls
['may'] ['2019']
data\monthly-portfolio-as-on-february-2019.xls
['february'] ['2019']
data\monthly-portfolio-as-on-january-2019.xls
['january'] ['2019']
data\monthly_portfolio_in_half_yearly_format_as_on_30_sept_2019.xls
[] ['2019']


In [15]:
month_order = OrderedDict()
month_order['january']= 1
month_order['february']= 2
month_order['march']= 3
month_order['april']= 4
month_order['may']= 5
month_order['june']= 6
month_order['july']= 7
month_order['august']= 8
month_order['september']= 9
month_order['october']= 10
month_order['november']= 11
month_order['december']= 12

In [16]:
df_new = pd.ExcelFile('data//monthly-portfolio-as-on-29th-february-2020.xls')
df_old = pd.ExcelFile('data//monthly-portfolio-as-on-31st-january-2020.xls')

In [17]:
sheet_new = df_new.sheet_names
sheet_old = df_old.sheet_names

In [18]:
date_pat1 = re.compile(r'\d\d-\d\d-\d{2,4}')
date_pat2 = re.compile(r'\d\d/\d\d/\d{2,4}')
def date_extract(df):
    date_loc = df.iloc[3:4,1]
    date_loc1 = re.findall(date_pat1, str(date_loc))
    date_loc2 = re.findall(date_pat2, str(date_loc))
    if len(date_loc1) != 0:
        logger.debug("%s",date_loc1)
        date = datetime.datetime.strptime(date_loc1[0], "%d-%m-%y")
        return date
    elif len(date_loc2) != 0:
        logger.debug("%s",date_loc2)
        date = datetime.datetime.strptime(date_loc2[0], "%d/%m/%y")
        return date
    else:
        date = 'Null'
        return date

In [95]:
def create_df(df, month_date):
    qty = 'Quantity_'+month_date
    mkt_val = 'Market_val_'+month_date
    nav = 'pnav_'+month_date
    dftmp = df.iloc[8:,1:7]
    dftmp = dftmp.rename(columns = {df.columns[1]:'Stock_Name', 'Unnamed: 2' : 'INDUSTRY', 'Unnamed: 3' : 'ISIN Code',
       'Unnamed: 4' : qty, 'Unnamed: 5' : mkt_val, 'Unnamed: 6': nav })
    return dftmp, qty, mkt_val, nav

In [61]:
def find_change(new, old):
    try:
        old = int(old)
        new = int(new)
        if old==0:
            return "New"
        elif new == 0:
            return "Sold"
        elif old > new:
            return "Decrease"
        elif old == new:
            return "No Change"
        elif old < new:
            return "Increase"
    except Exception:
        return "NA"

In [60]:
def find_act_change(new, old):
    try:
        old = int(old)
        new = int(new)
        if old == 0:
            return 100
        elif new == 0:
            return 0
        else:
            chg = abs(new - old)*100/ old
            return int(chg)
    except Exception:
        return "NA"

In [78]:
try:
    sheet_name = 'Index'
    df_temp1 = df_new.parse(sheet_name)
    df_temp2 = df_old.parse(sheet_name)
    df_temp1 = df_temp1[df_temp1['Index'] == 'Equity']
    df_temp2 = df_temp2[df_temp2['Index'] == 'Equity']
    sheet_names_new = df_temp1['Unnamed: 1'].tolist()
    sheet_names_old = df_temp2['Unnamed: 1'].tolist()
except Exception:
    print("Error")

In [98]:
writer = pd.ExcelWriter('MutualFund_Analysis.xlsx', mode='w')
for name in sheet_names_new:
    if name in sheet_names_old:
        print(name)
        df1 = df_new.parse(name)
        df2 = df_old.parse(name)
        date1 = date_extract(df1)
        month_date1 = str(date1.month)+'_'+str(date1.day)
        logging.debug(" Month df1 : %s",month_year1)
        date2 = date_extract(df2)
        month_date2 = str(date2.month)+'_'+str(date2.day)
        logging.debug(" Month df2 : %s",month_year2)
        assert(date1.month - date2.month == 1)
        logging.debug("Assertion correct")
        dftmp1, qty1, mkt1, pnav1 = create_df(df1,month_date1)
        dftmp2, qty2, mkt2, pnav2 = create_df(df2,month_date2)
        logging.debug("new df created")
        dftmpx1 = dftmp1[dftmp1['INDUSTRY'].notna()]
        logger.debug("df1_shape : %s",dftmpx1.shape)
        dftmpx2 = dftmp2[dftmp2['INDUSTRY'].notna()]
        logger.debug("df2_shape : %s",dftmpx2.shape)
        dftmpx1 = dftmp1[dftmp1['ISIN Code'].notna()]
        print(dftmpx1.columns)
        dftmpx2 = dftmp2[dftmp2['ISIN Code'].notna()]
        #dftmpx1 = dftmp1[dftmp1[nav].notna()]
        #dftmpx2 = dftmp2[dftmp2[nav].notna()]
        dftmp = pd.merge(dftmpx1, dftmpx2, how = 'outer', on = ['Stock_Name', 'INDUSTRY', 'ISIN Code'])
        dftmp = dftmp.replace(np.nan, 0)
        dftmp['change'] = dftmp[[qty1, qty2]].apply(lambda x: find_change(x[0],x[1]), axis=1)
        dftmp['change_val'] = dftmp[[qty1, qty2]].apply(lambda x: find_act_change(x[0],x[1]), axis=1)
        dftmp.to_excel(writer, sheet_name=name)
writer.save()        

DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (693, 6)
DEBUG:root:df2_shape : (709, 6)


TAF
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TBAF


DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (488, 6)
DEBUG:root:df2_shape : (410, 6)
DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (18, 6)
DEBUG:root:df2_shape : (19, 6)


Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TBFSF
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TDIF

DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (10, 6)
DEBUG:root:df2_shape : (10, 6)
DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (40, 6)
DEBUG:root:df2_shape : (41, 6)
DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (266, 6)


...

TNPBETF
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TQNTF
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TREF
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TRSFM

DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (67, 6)
DEBUG:root:df2_shape : (64, 6)
DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (49, 6)
DEBUG:root:df2_shape : (49, 6)
DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (36, 6)
DEBUG:root:df2_shape : (36, 6)



Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TRSFP
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TSCAPF
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')


DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (45, 6)
DEBUG:root:df2_shape : (42, 6)
DEBUG:root:['29/02/20']
DEBUG:root: Month df1 : 2_29
DEBUG:root:['31-01-20']
DEBUG:root: Month df2 : 1_31
DEBUG:root:Assertion correct
DEBUG:root:new df created
DEBUG:root:df1_shape : (40, 6)
DEBUG:root:df2_shape : (39, 6)


TVFS1
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')
TVFS2
Index(['Stock_Name', 'INDUSTRY', 'ISIN Code', 'Quantity_2_29',
       'Market_val_2_29', 'pnav_2_29'],
      dtype='object')


In [63]:
dftmp

,Stock_Name,INDUSTRY,ISIN Code,Quantity_2_29,Market_val_2_29,pnav_2_29,Quantity_1_31,Market_val_1_31,pnav_1_31,change,change_val
0,HDFC BANK LTD,BANKS,INE040A01034,22856,269.16,11.002596,20580,252.37,10.801806,Increase,11
1,RELIANCE INDUSTRIES LTD,PETROLEUM PRODUCTS,INE002A01018,17155,227.93,9.317084,15454,218.16,9.337319,Increase,11
2,HOUSING DEVELOPMENT FINANCE CORPORATION LTD,FINANCE,INE001A01036,9121,198.45,8.112042,8213,198.26,8.485811,Increase,11
3,ICICI BANK LTD,BANKS,INE090A01021,33892,168.53,6.888913,30489,160.27,6.859526,Increase,11
4,INFOSYS LTD,SOFTWARE,INE009A01021,19276,141.04,5.765390,17350,134.63,5.762189,Increase,11
5,KOTAK MAHINDRA BANK LTD,BANKS,INE237A01028,7095,114.96,4.699373,6392,108.14,4.628361,Increase,10
6,TATA CONSULTANCY SERVICES LTD,SOFTWARE,INE467B01029,5512,110.25,4.506615,4960,103.12,4.413680,Increase,11
7,ITC LTD,CONSUMER NON DURABLES,INE154A01025,45985,90.84,3.713402,41395,97.34,4.166267,Increase,11
8,HINDUSTAN UNILEVER LTD,CONSUMER NON DURABLES,INE030A01027,3742,81.38,3.326533,3366,68.47,2.930709,Increase,11
9,AXIS BANK LTD,BANKS,INE238A01034,11668,81.36,3.325790,10495,76.54,3.275993,Increase,11


In [47]:
df_temp = df1.copy()
date = df_temp.iloc[3:4,1]
date
date1 = re.sub(r'[a-zA-Z]',"",str(date))
pat = re.compile(r'\d\d[-/]\d\d[-/]\d{2,4}')
date
date2 = re.search(date_pat2,str(date))
date2.group()
date3 = date2.group()
date4 = datetime.datetime.strptime(date3, "%d/%m/%y")
x = re.search('/',str(date))

3    as on 29/02/20
Name: TATA ARBITRAGE FUND, dtype: object

In [100]:
import numpy as np
import pandas as pd
import re

In [101]:
df1 = pd.read_excel('./data/monthly-portfolio-as-on-31st-january-2020.xls', sheet_name='TEQPEF')

FileNotFoundError: [Errno 2] No such file or directory: './data/monthly-portfolio-as-on-31st-january-2020.xls'

In [4]:
df2 = pd.read_excel('./data/monthly-portfolio-as-on-29th-february-2020.xls', sheet_name='TEQPEF')

In [43]:
month_year = '012020'
qty = 'Quantity_'+month_year

In [24]:
def create_df(df, month_date):
    qty = 'Quantity_'+month_date
    mkt_val = 'Market_val_'+month_date
    nav = 'pnav_'+month_date
    dftmp = df.iloc[8:,1:7]
    dftmp = dftmp.rename(columns = {df.columns[1]:'Stock_Name', 'Unnamed: 2' : 'INDUSTRY', 'Unnamed: 3' : 'ISIN Code',
       'Unnamed: 4' : qty, 'Unnamed: 5' : mkt_val, 'Unnamed: 6': nav })
    return dftmp

In [7]:
dftmp1 = create_df(df1,'012020')
dftmp2 = create_df(df2,'022020')

In [8]:
dftmpx1 = dftmp1[dftmp1['INDUSTRY'].notna()]
dftmpx2 = dftmp2[dftmp2['INDUSTRY'].notna()]

In [19]:
dftmp = pd.merge(dftmpx1, dftmpx2, how = 'outer', on = ['Stock_Name', 'INDUSTRY', 'ISIN Code'])
dftmp = dftmp.replace(np.nan, 0)

In [20]:
def find_change(old, new):
    old = int(old)
    new = int(new)
    if old==0:
        return "New"
    elif new == 0:
        return "Sold"
    elif old > new:
        return "Decrease"
    elif old == new:
        return "No Change"
    elif old < new:
        return "Increase"


In [40]:
def find_act_change(old, new):
    old = int(old)
    new = int(new)
    if old == 0:
        return 100
    elif new == 0:
        return 0
    else:
        chg = abs(new - old)*100/ old
        return int(chg)

In [21]:
dftmp['change'] = dftmp[['Quantity_012020', 'Quantity_022020']].apply(lambda x: find_change(x[0],x[1]), axis=1)

In [39]:
dftmp['change_val'] = dftmp[['Quantity_012020','Quantity_022020']].apply(lambda x: find_act_changect_change(x[0],x[1]), axis=1)

1990000
1990000
2900000
2850000
6730000
6400000
14600000
7200000
2385000
3135000
1400000
600000
525000
525000
3550000
3150000
23000
20952
908976
908976
4000000
4000000
5400000
4900000
709746
709746
650000
650000
1375000
1325000
2800000
2600000
12400000
12400000
2120000
2120000
470000
0
2170000
2170000
720000
755000
4883866
4883866
3635000
3685000
1028200
1028200
708000
708000
821000
1271000
1629287
1629287
1000000
1000000
26000
26000
2950000
2950000
293000
293000
1059750
1059750
170000
170000
1631000
1446000
710000
710000
165000
165000
389000
389000
1281768
231768
630000
1000000
1869763
0
1108508
1108508
0
1460000


In [22]:
dftmpx1

,Stock_Name,INDUSTRY,ISIN Code,Quantity_012020,Market_val_012020,pnav_012020
8,HOUSING DEVELOPMENT FINANCE CORPORATION LTD,FINANCE,INE001A01036,1990000,48038.6,9.49227
9,HDFC BANK LTD,BANKS,INE040A01034,2900000,35562.7,7.02707
10,ICICI BANK LTD,BANKS,INE090A01021,6730000,35376.2,6.99023
11,ITC LTD,CONSUMER NON DURABLES,INE154A01025,14600000,34331.9,6.78387
12,RELIANCE INDUSTRIES LTD,PETROLEUM PRODUCTS,INE002A01018,2385000,33667.8,6.65265
13,LARSEN & TOUBRO LTD,CONSTRUCTION PROJECT,INE018A01030,1400000,19170.2,3.78797
14,BAJAJ AUTO LTD,AUTO,INE917I01010,525000,16695.3,3.29893
15,BHARAT PETROLEUM CORPORATION LTD,PETROLEUM PRODUCTS,INE029A01011,3550000,16221.7,3.20536
16,MRF LTD,AUTO ANCILLARIES,INE883A01011,23000,16025.2,3.16652
17,ACC LTD,CEMENT,INE012A01025,908976,13676.5,2.70242


In [23]:
dftmpx2

,Stock_Name,INDUSTRY,ISIN Code,Quantity_022020,Market_val_022020,pnav_022020
8,HOUSING DEVELOPMENT FINANCE CORPORATION LTD,FINANCE,INE001A01036,1990000,43297.4,9.48005
9,RELIANCE INDUSTRIES LTD,PETROLEUM PRODUCTS,INE002A01018,3135000,41653.2,9.12004
10,HDFC BANK LTD,BANKS,INE040A01034,2850000,33563,7.34869
11,ICICI BANK LTD,BANKS,INE090A01021,6400000,31824,6.96792
12,BAJAJ AUTO LTD,AUTO,INE917I01010,525000,15172.5,3.32205
13,ITC LTD,CONSUMER NON DURABLES,INE154A01025,7200000,14223.6,3.11428
14,MRF LTD,AUTO ANCILLARIES,INE883A01011,20952,13910.7,3.04577
15,BHARAT PETROLEUM CORPORATION LTD,PETROLEUM PRODUCTS,INE029A01011,3150000,13430,2.94053
16,STATE BANK OF INDIA,BANKS,INE062A01020,4000000,12120,2.6537
17,ACC LTD,CEMENT,INE012A01025,908976,12006.2,2.62878


In [41]:
dftmp

,Stock_Name,INDUSTRY,ISIN Code,Quantity_012020,Market_val_012020,pnav_012020,Quantity_022020,Market_val_022020,pnav_022020,change,change_val
0,HOUSING DEVELOPMENT FINANCE CORPORATION LTD,FINANCE,INE001A01036,1990000,48038.60,9.492265,1990000,43297.43,9.480052,No Change,0
1,HDFC BANK LTD,BANKS,INE040A01034,2900000,35562.70,7.027070,2850000,33563.03,7.348687,Decrease,1
2,ICICI BANK LTD,BANKS,INE090A01021,6730000,35376.25,6.990227,6400000,31824.00,6.967924,Decrease,4
3,ITC LTD,CONSUMER NON DURABLES,INE154A01025,14600000,34331.90,6.783868,7200000,14223.60,3.114284,Decrease,50
4,RELIANCE INDUSTRIES LTD,PETROLEUM PRODUCTS,INE002A01018,2385000,33667.85,6.652654,3135000,41653.18,9.120041,Increase,31
5,LARSEN & TOUBRO LTD,CONSTRUCTION PROJECT,INE018A01030,1400000,19170.20,3.787967,600000,7125.30,1.560098,Decrease,57
6,BAJAJ AUTO LTD,AUTO,INE917I01010,525000,16695.26,3.298928,525000,15172.50,3.322047,No Change,0
7,BHARAT PETROLEUM CORPORATION LTD,PETROLEUM PRODUCTS,INE029A01011,3550000,16221.73,3.205358,3150000,13430.03,2.940529,Decrease,11
8,MRF LTD,AUTO ANCILLARIES,INE883A01011,23000,16025.19,3.166524,20952,13910.70,3.045774,Decrease,8
9,ACC LTD,CEMENT,INE012A01025,908976,13676.45,2.702421,908976,12006.21,2.628782,No Change,0


In [64]:
sheet_new

['Index',
 'TAF',
 'TBAF',
 'TMIF',
 'TLF',
 'TSTBF',
 'TTMFSP1',
 'TIPF',
 'TFF',
 'TFRSTF',
 'TVFS1',
 'TVFS2',
 'TSCAPF',
 'TRSFP',
 'TRSFM',
 'TREF',
 'TQNTF',
 'TNPBETF',
 'TNETF',
 'TMCAPF',
 'TINR',
 'TTOFE',
 'TEOF',
 'TISF',
 'TTSF96',
 'TIPHF',
 'TICF',
 'TIFSA',
 'TIFNA',
 'TFEF',
 'TCS',
 'TEQPEF',
 'TDIF',
 'TBFSF',
 'TUSTF',
 'TRSFC',
 'TOF',
 'TIFA',
 'TGSFR',
 'TDBF',
 'TBPSUF',
 'TYCF',
 'TEGF',
 'TFM53A',
 'TFM53B',
 'TFM54A',
 'TFM55A',
 'TFM55B',
 'TFM55D',
 'TFM55E',
 'TFM55F',
 'TFM55G',
 'TFM55I',
 'TFM56A',
 'TFM56B',
 'TFM56C',
 'TFM56D',
 'TFM56F']

In [65]:
df_temp = df_new.parse(sheet_new[0])
df_temp = df_temp[df_temp['Index'] == 'Equity']
sheet_names = df_temp['Unnamed: 1'].tolist()

In [73]:
df_temp.head()

,Index,Unnamed: 1,Unnamed: 2
30,Equity,TAF,TATA ARBITRAGE FUND
31,Equity,TBAF,TATA BALANCED ADVANTAGE FUND
32,Equity,TBFSF,TATA BANKING & FINANCIAL SERVICES FUND
33,Equity,TDIF,TATA DIGITAL INDIA FUND
34,Equity,TEQPEF,TATA EQUITY P/E FUND


In [107]:
df1 = pd.read_excel('./data_all/monthly-portfolio-as-on-29th-february-2020.xls', sheet_name='TAF')

In [103]:
df1 = create_df(df1, '022020')

In [108]:
dftmp = df1.iloc[8:,1:7]

In [111]:
dftmp = dftmp.rename(columns = {'Unnamed: 3' : 'ISIN'})

In [112]:
dftmp['ISIN']

8      INE397D01024
9      INE062A01020
10     INE018A01030
11     INE154A01025
12     INE090A01021
           ...     
737             NaN
738             NaN
739             NaN
740             NaN
741             NaN
Name: ISIN, Length: 734, dtype: object

In [ ]:
isin_pat = re.compile(r'I')

In [123]:
def isincheck(data):
    print(data)
    try:
        if data.startswith('IN'):
            print(data)
            return "yes"
        else:
            return "no"
    except Exception:
        print("except")
        return "no"

In [124]:
dftmp['ISIN_check'] = dftmp[['ISIN']].apply(lambda x : isincheck(x[0]), axis=1)

INE397D01024
INE397D01024
INE062A01020
INE062A01020
INE018A01030
INE018A01030
INE154A01025
INE154A01025
INE090A01021
INE090A01021
INE854D01024
INE854D01024
INE047A01021
INE047A01021
INE095A01012
INE095A01012
INE001A01036
INE001A01036
INE585B01010
INE585B01010
INE238A01034
INE238A01034
INE009A01021
INE009A01021
INE171A01029
INE171A01029
INE776C01039
INE776C01039
INE271C01023
INE271C01023
INE038A01020
INE038A01020
INE158A01026
INE158A01026
INE044A01036
INE044A01036
INE192A01025
INE192A01025
INE089A01023
INE089A01023
INE814H01011
INE814H01011
INE476A01014
INE476A01014
INE040A01034
INE040A01034
INE102D01028
INE102D01028
INE424H01027
INE424H01027
INE917I01010
INE917I01010
INE988K01017
INE988K01017
INE318A01026
INE318A01026
INE237A01028
INE237A01028
INE280A01028
INE280A01028
INE795G01014
INE795G01014
INE055A01016
INE055A01016
INE742F01042
INE742F01042
INE176A01028
INE176A01028
INE628A01036
INE628A01036
INE481G01011
INE481G01011
INE012A01025
INE012A01025
INE030A01027
INE030A01027
INE021A01026

nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except
nan
except


In [126]:
dftmp[dftmp['ISIN_check'] == 'yes']

,TATA ARBITRAGE FUND,Unnamed: 2,ISIN,Unnamed: 4,Unnamed: 5,Unnamed: 6,ISIN_check
8,BHARTI AIRTEL LTD,TELECOM - SERVICES,INE397D01024,323925,1695.75,2.47746,yes
9,STATE BANK OF INDIA,BANKS,INE062A01020,528000,1599.84,2.33734,yes
10,LARSEN & TOUBRO LTD,CONSTRUCTION PROJECT,INE018A01030,118875,1411.7,2.06247,yes
11,ITC LTD,CONSUMER NON DURABLES,INE154A01025,703200,1389.17,2.02956,yes
12,ICICI BANK LTD,BANKS,INE090A01021,264000,1312.74,1.9179,yes
...,...,...,...,...,...,...,...
183,** 08.75 % LIC HOUSING FINANCE LTD - 21/12/2020,CRISIL- AAA,INE115A07JC2,60,610.42,0.891814,yes
184,** 08.50 % REC LTD - 20/12/2021,CRISIL- AAA,INE020B08BR5,50,516.52,0.754626,yes
187,** - ICICI BANK LTD - CD - 14/09/2020,ICRA- A1+,INE090A166V8,1000,970.15,1.41738,yes
188,** - KOTAK MAHINDRA INVESTMENTS LTD - CP - 30/...,CRISIL- A1+,INE975F14RX3,100,495.14,0.723394,yes
